## 3.1.6 Realistic simulation setup
In this script it is solved the problem of the outter circle pixels not being taken into consideration. The problem occurs when the circle passes into another pixel but doesn't touch any corners. The previous method didn't account that problem. The results has a discretization missing some minor parts of the circle.

### Plot spot with pixel diameter
- ~~create a mesh with pixel size distance corners~~
- ~~create a spot centered in the middle of the plane~~
- ~~Pass the spot through the mesh~~ 
- ~~Each pixel that has the spot goes to a micro mesh~~
- ~~Each micropixel that has the spot sum.~~
- ~~Consider the case where the spot is inbetween two pixels~~
    - ~~Expand the mesh by 1 in each direction~~
- ~~Consider the case where the spot is inbetween 4 pixels~~
- Consider the case where:
    - ~~spot is double the size centered inbetween  4 pixels~~
    ~~- spot is centered in the middle of a pixel~~
- Approach where the inner and outter circle parts are taken into consideration

### import  libraries

In [202]:
vscode = 1

In [203]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation



DECIMALS = 2            # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

### Spot 

In [204]:
#spot centroid coordinates 
cx0 = 5.675
cy0 = 3.5630

In [205]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

In [206]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [207]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):
    # array for percentage storage
    pixel_covered_percent = np.zeros((1,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width

    x_pixel = cx0max+xgrid
    y_pixel = cy0max+ygrid

    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
    circ = np.where(circ_total == True)

    pixel_covered = circ[0].shape
    totalpixel_area = gridSize**2
    pixel_covered_percent = (pixel_covered[0]/totalpixel_area)*100
    #print("pixel cover percentage: " + str(pixel_covered_percent) + " %")

    return pixel_covered_percent

In [208]:
# the objective is, knowing the radius, create an area to be inspected. The difficulty stands in having the whole object in the area.
cx0max,cx0min,cy0max,cy0min = spot_center_coord(cx0,cy0,pixelGridX,pixelGridY)

In [210]:
gridSize = 1024
theta = np.linspace(0, 2*np.pi, gridSize)
zoom = 1/3
r = pixel_width*zoom
x0 = r*np.cos(theta)+(cx0)
y0 = r*np.sin(theta)+(cy0)

fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=1)
#ax.plot(pixelGridX[circ_corners[1][:]],pixelGridY[circ_corners[0][:]],'r*')
ax.plot(cx0,cy0,'k.')
ax.set_xlim([cx0-(zoom*pixel_width),cx0+(zoom*pixel_width)])
ax.set_ylim([cy0-(zoom*pixel_width),cy0+(zoom*pixel_width)])
ax.set_aspect(1)
plt.show()

In [212]:
cx0max_pxl = []
cx0min_pxl = []
cy0max_pxl = []
cy0min_pxl = []
points_max_min = []
for i in range(x0.shape[0]):
    cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0[i],y0[i],pixelGridX,pixelGridY)

    # max and min points in (x,y) coordinates
    cx0min_pxl.append(cx0min*pixel_width)
    cx0max_pxl.append(cx0max*pixel_width)
    cy0min_pxl.append(cy0min*pixel_width)
    cy0max_pxl.append(cy0max*pixel_width)

    #array max and min points
    points_max_min.append(([cx0min_pxl[i],cy0min_pxl[i]]))

### Remove outter circle duplicates

In [213]:
#list for the final values
points_deduplt = list()

[points_deduplt.append(item) for item in points_max_min if item not in points_deduplt]
print(np.shape(points_deduplt))

(2, 2)


## Inner circle corners

In [214]:
circ1 = np.sqrt((pixelGridXX-cx0)**2 + (pixelGridYY-cy0)**2) <= r
circ_corners = np.where(circ1 == True)
circ_corners = np.array(circ_corners)
# transpose and obtain the values in mm
circ_corners_arr = (circ_corners*pixel_width).T
# swap the coordinates(for better readability)
circ_corners_arr = np.array([circ_corners_arr[:,1],circ_corners_arr[:,0]]).T
# revert to list
circ_corners_list = circ_corners_arr.tolist()
np.shape(circ_corners_list)

(0,)

### Remove outter and inner circle duplicates

In [215]:
# join both lists
circle_list = points_deduplt + circ_corners_list
#list for the final values
circle_list_deduplt = list()

[circle_list_deduplt.append(item) for item in circle_list if item not in circle_list_deduplt]
print(np.shape(circle_list_deduplt)) 

(2, 2)


## Plot the results

In [201]:
fig,ax = plt.subplots()

# center of the pixel
ax.plot(cx0,cy0,'k.')
#axis
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
#inner corners
ax.plot(circ_corners_arr[:,0],circ_corners_arr[:,1],'r*')
#spot
ax.plot(x0,y0,'r',linewidth=4)
# list for the patches
ptchs =[]
#pixel cover list
pixel_covered_percent = []
for j in range(len(circle_list_deduplt)):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,circle_list_deduplt[j][0],circle_list_deduplt[j][1],cx0,cy0,r))
    ptchs.append(Rectangle((circle_list_deduplt[j][0],circle_list_deduplt[j][1]), pixel_width,pixel_width))

p = PatchCollection(ptchs, cmap=cm.jet, alpha=1)
pixel_covered_percent_arr = np.array(pixel_covered_percent).flatten()
# define the color
p.set_array(pixel_covered_percent_arr)
# set color max and min
p.set_clim([0, 100])
ax.add_collection(p)

fig.colorbar(p)

ax.set_xlim([cx0-((zoom+1)*pixel_width),cx0+((zoom+1)*pixel_width)])
ax.set_ylim([cy0-((zoom+1)*pixel_width),cy0+((zoom+1)*pixel_width)])

ax.set_aspect(1)

plt.show()